In [13]:
# Import Libraries

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import glob
import re
import os
import pickle

# import nltk 
# nltk.download('stopwords')
# nltk.download('wordnet')
# from nltk.corpus import stopwords
# from nltk.stem import WordNetLemmatizer

# from wordcloud import WordCloud

# from sklearn.model_selection import train_test_split

# from keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequences

# import gensim.models.keyedvectors as word2vec

# from tensorflow.keras.layers import Embedding,Dense,LSTM,Bidirectional
# from tensorflow.keras.layers import BatchNormalization,Dropout

# from tensorflow.keras import Sequential


# from keras.models import load_model

In [15]:
# path = r'/Users/user/Desktop/Tweet Data/Tweeter Scrapping'
# all_files = glob.glob(os.path.join(path, "*.csv"))

# df = (pd.read_csv(f) for f in all_files)
# tweet_all  = pd.concat(df, ignore_index=True)
# tweet_all

In [14]:
path = r'Tweets/Tweet_All.csv'
tweet_all = pd.read_csv(path)
tweet_all

,Username_Query,Datetime,Tweet Id,Text
0,kevinhart4real,2021-01-22 02:41:31,1.352446e+18,😂😂😂😂😂😂 https://t.co/S6bufs38c6
1,kevinhart4real,2021-01-21 21:00:42,1.352360e+18,Mood on this beautiful Thursday.... #LiveLoveL...
2,kevinhart4real,2021-01-21 16:47:20,1.352297e+18,Just posted a photo https://t.co/QA4YrxAD39
3,kevinhart4real,2021-01-20 01:18:07,1.351701e+18,For Celebrity Game Face tonight: \n\nThese cou...
4,kevinhart4real,2021-01-19 21:20:56,1.351641e+18,Haters will say that I’m not tall..... 🤷🏾‍♂️ s...
...,...,...,...,...
43193,riceuniversity,2019-07-18 15:49:29,1.151882e+18,RT @ScottESolomon: Great to see this at @Moody...
43194,riceuniversity,2019-07-18 15:08:59,1.151871e+18,@scifiya7th @RiceU_STEM @conocophillips Yes of...
43195,riceuniversity,2019-07-18 14:24:17,1.151860e+18,Space veterans who worked on Apollo 11 joined ...
43196,riceuniversity,2019-07-18 14:04:11,1.151855e+18,RT @NASA_Johnson: President Kennedy’s challeng...


In [15]:
#tweet_all = tweet_all[tweet_all.columns.drop([ 'Datetime'])]
tweet_all = tweet_all.drop_duplicates(subset=['Text'], keep='first').reset_index(drop=True)
#tweet_all = tweet_all.reset_index(drop=True)
tweet_all

,Username_Query,Datetime,Tweet Id,Text
0,kevinhart4real,2021-01-22 02:41:31,1.352446e+18,😂😂😂😂😂😂 https://t.co/S6bufs38c6
1,kevinhart4real,2021-01-21 21:00:42,1.352360e+18,Mood on this beautiful Thursday.... #LiveLoveL...
2,kevinhart4real,2021-01-21 16:47:20,1.352297e+18,Just posted a photo https://t.co/QA4YrxAD39
3,kevinhart4real,2021-01-20 01:18:07,1.351701e+18,For Celebrity Game Face tonight: \n\nThese cou...
4,kevinhart4real,2021-01-19 21:20:56,1.351641e+18,Haters will say that I’m not tall..... 🤷🏾‍♂️ s...
...,...,...,...,...
21558,riceuniversity,2019-07-18 15:49:29,1.151882e+18,RT @ScottESolomon: Great to see this at @Moody...
21559,riceuniversity,2019-07-18 15:08:59,1.151871e+18,@scifiya7th @RiceU_STEM @conocophillips Yes of...
21560,riceuniversity,2019-07-18 14:24:17,1.151860e+18,Space veterans who worked on Apollo 11 joined ...
21561,riceuniversity,2019-07-18 14:04:11,1.151855e+18,RT @NASA_Johnson: President Kennedy’s challeng...


In [17]:
model = load_model('../Model1/Twitter-Sentiment-BiLSTM.h5')
model.summary()

Model: "Sentiment_Model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 60, 100)           27283600  
_________________________________________________________________
bidirectional (Bidirectional (None, 60, 200)           160800    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 60, 200)           240800    
_________________________________________________________________
conv1d (Conv1D)              (None, 56, 100)           100100    
_________________________________________________________________
global_max_pooling1d (Global (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 16)                1616      
_________________________________________________________________
dense_1 (Dense)              (None, 1)             

In [18]:
file = open('../Model1/Tokenizer.pickle', 'rb')
tokens = pickle.load(file)
file.close()

In [29]:
text = list(tweet_all.Text)#[0:100]
#text = ['I love spring', 'I have winter']

In [30]:
%%time
max_length = 20
trunc_type = 'post'
padd_type = 'post'


# Predicting

def decode_sentiment(score):
    label = None
    if score <= 0.5:
        label = 'Negative'
    else:
        label = 'Positive'
    
    return label

def predict(text):
    
        
    x_test = pad_sequences(tokens.texts_to_sequences([text]), maxlen=max_length,padding=padd_type,truncating=trunc_type)
    score = model.predict([x_test])
    label = decode_sentiment(score)
    
    return  {"text": text, "label": label, "score": float(score)}

df = pd.DataFrame()


for i in text:
    df = df.append(predict(i), ignore_index=True)


df['User'] = tweet_all['Username_Query'].values#[0:100]
df['tweet_id'] = tweet_all['Tweet Id'].values#[0:100]
df['datetime'] = tweet_all['Datetime'].values#[0:100]
df.to_csv('../Model1/model1_test.csv', encoding='utf-8', index=False)
df.head(300)


CPU times: user 22min 6s, sys: 1min 24s, total: 23min 30s
Wall time: 18min 44s


,label,score,text,User,tweet_id,datetime
0,Positive,0.515525,😂😂😂😂😂😂 https://t.co/S6bufs38c6,kevinhart4real,1.352446e+18,2021-01-22 02:41:31
1,Positive,0.709734,Mood on this beautiful Thursday.... #LiveLoveL...,kevinhart4real,1.352360e+18,2021-01-21 21:00:42
2,Positive,0.664074,Just posted a photo https://t.co/QA4YrxAD39,kevinhart4real,1.352297e+18,2021-01-21 16:47:20
3,Positive,0.972216,For Celebrity Game Face tonight: \n\nThese cou...,kevinhart4real,1.351701e+18,2021-01-20 01:18:07
4,Positive,0.634962,Haters will say that I’m not tall..... 🤷🏾‍♂️ s...,kevinhart4real,1.351641e+18,2021-01-19 21:20:56
...,...,...,...,...,...,...
295,Positive,0.515525,#zerofsgiven https://t.co/jQQrWV5IOD,kevinhart4real,1.329422e+18,2020-11-19 13:51:49
296,Positive,0.515525,#zerofsgiven https://t.co/bFWb3tqrhi,kevinhart4real,1.329422e+18,2020-11-19 13:51:33
297,Positive,0.515525,😂😂😂😂 #ZeroFucksGiven https://t.co/RmwYJcTfWm,kevinhart4real,1.329422e+18,2020-11-19 13:51:00
298,Positive,0.515525,#ZeroFucksGiven https://t.co/vHeXIjvp8a,kevinhart4real,1.329422e+18,2020-11-19 13:50:05
